In [1]:
import transformers 
import datasets
import torch
import logging
import json
import pandas as pd
from torch.utils.data import Subset

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MyDataset:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data[index]
        return {
            'id': row['id'],
            'question': row['question'],
            'answer': row['answer'],
            'context': row['context'],
            'answer_start': row['answer_start'],
            'answer_end': row['answer_end']
        }

    def get(self, index):
        return self.__getitem__(index)

    def set(self, index, id=None, question=None, answer=None, context=None, answer_start=None, answer_end=None):
        if id is not None:
            self.data[index]['id'] = id
        if question is not None:
            self.data[index]['question'] = question
        if answer is not None:
            self.data[index]['answer'] = answer
        if context is not None:
            self.data[index]['context'] = context
        if answer_start is not None:
            self.data[index]['answer_start'] = answer_start
        if answer_end is not None:
            self.data[index]['answer_end'] = answer_end

In [3]:
dataset=torch.load("../datasets/ms-marco_train_qa.pt")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'context', 'answer_start', 'answer_end'],
        num_rows: 301763
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'context', 'answer_start', 'answer_end'],
        num_rows: 201176
    })
})

In [4]:
dataset=dataset.train_test_split(test_size=0.4)

In [5]:
dataset_train=dataset['train']

In [6]:
dataset_train[0]

{'id': 862709,
 'question': 'what is whatsapp executable',
 'answer': 'Known as BlackBerry Desktop Software and it is developed by Research In Motion.',
 'context': "Start FreeFixer and scan your computer again. If WhatsApp.exe still remains in the scan result, proceed with the next step. If WhatsApp.exe is gone from the scan result you're done. If WhatsApp.exe still remains in the scan result, check its checkbox again in the scan result and click Fix. Whatsapp.exe for pc social advice. Users interested in Whatsapp.exe for pc generally download: Free. Wassapp is a PC application developed to be a non-official client for WhatsApp Messenger. You just have to type your phone number, import your contacts and chat with them instantly using your keyboard. The application has a simple and easy to use interface. What is WhatsApp .exe ? WhatsApp .exe is known as BlackBerry Desktop Software and it is developed by Research In Motion , it is also developed by . We have seen about 2 different insta

In [7]:
dataset=dataset['test'].train_test_split(test_size=0.5)

In [8]:
dataset_valid=dataset['train']

In [9]:
dataset_valid[0]

{'id': 575498,
 'question': 'what are the uses of juniper berries',
 'answer': 'As a spice used in a wide variety of culinary dishes, as a primary flavoring in gin.Also used as the primary flavor in the liquor Jenever and sahti-style of beers.Juniper berry sauce is often a popular flavoring choice.',
 'context': "Comments. Submit. Juniper berries have been in use since the time of the Greeks. They have had many uses from medicinal to culinary for hundreds of years. If you have access to this interesting spice, you can use juniper berries in a number of different ways, from seasoning food to being used as a diuretic.However, fresh juniper berries can be dangerous to ingest for those with kidney problems or those who are pregnant.f you have access to this interesting spice, you can use juniper berries in a number of different ways, from seasoning food to being used as a diuretic. However, fresh juniper berries can be dangerous to ingest for those with kidney problems or those who are pre

In [10]:
dataset_test=dataset['test']

In [12]:
dataset_test[0]

{'id': 770053,
 'question': 'what is meant by the phrase the reflective practitioner',
 'answer': 'The capacity to reflect on action so as to engage in a process of continuous learning. According to one definition it involves paying critical attention to the practical values and theories which inform everyday actions, by examining practice reflectively and reflexively.',
 'context': "Give an overview of how reflection is used in nursing. Explain its relevance to nursing and how and when it is used. Explore the concepts of reflective practice and critical incident analysis. Introduce use of reflection for personal development. This page provides all possible meanings and translations of the word Reflective practice. Reflective practice. Reflective practice is the capacity to reflect on action so as to engage in a process of continuous learning. According to one definition it involves paying critical attention to the practical values and theories which inform everyday actions, by examini

In [4]:
squad=torch.load("../datasets/squad.pt")

In [14]:
squad['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [13]:
# calculate max context length for dataset
def calc_max_len(dataset):
  context_length_max=len(dataset[0]['context'])
  for i in range(len(dataset)):
    con_len=len(dataset[i]['context'])
    if(con_len<context_length_max):
      context_length_max=con_len
      print(context_length_max)
      print(dataset[i]['context'])
  return context_length_max


In [14]:
calc_max_len(dataset_train)

2769
Excess mucus in the stool is sometimes accompanied by other symptoms, which may be a sign of a bigger problem. These symptoms include: blood or pus in the stool. abdominal pain, cramping, or bloating. changes in bowel movements or habits. Diarrhea is not always a sign of illness. For some children, the occasional bout may be normal, even if it contains mucus. The key is to know when diarrhea is a symptom of a problem. Factors to evaluate include the frequency and consistency of the stool. A large increase in the amount of mucus in your child's feces may point to illness, however, especially if it is accompanied by loose stool or blood. The combination of watery stool and mucus may indicate a viral infection or medical problem with the intestines. Blood & Mucus: Work-up should be done to determine if you have inflammatory bowel disease(ulcerative colitis or crohn's ) depending upon the duration of the symptoms , there may of course be other explanation. Evaluation may include stool

301

In [16]:
# Preprocessing
# sep_token = '<sep>'
dataset_name = "ms_marco"
model_type="roberta"
model_name= "roberta-base"
models_dir = "saved_models/roberta-base_ms-marco_mod"
checkpoint = 'roberta-base'
max_input_length = 301


# ## Training
learning_rate = 3e-5
num_epochs = 2

In [17]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [27]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_input_length ,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    # answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        # answer = examples["answer_start"][i]
        
        start_char = examples["answer_start"][i]      
        end_char = examples["answer_end"][i]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# Transform your custom dataset to a PyTorch dataset
# dataset = datasets.Dataset.from_generator(
#     generator=lambda: iter(dataset)
# )

In [12]:
# dataset_val= datasets.Dataset.from_generator(
#     generator=lambda: iter(dataset_valid))

Dataset generator downloaded and prepared to C:/Users/dama_/.cache/huggingface/datasets/generator/default-1f482045682b315d/0.0.0. Subsequent calls will reuse this data.


In [16]:
# torch.save(dataset,"../datasets/ms-marco_train_qa.pt")

In [17]:
# torch.save(dataset_valid,"../datasets/ms-marco_valid_qa.pt")# 

In [29]:
dataset_train

Dataset({
    features: ['id', 'question', 'answer', 'context', 'answer_start', 'answer_end'],
    num_rows: 301763
})

In [38]:
dataset_train[0]

{'id': 584861,
 'question': 'what causes a rotator cuff tear',
 'answer': 'Bleeding and inflammation.',
 'context': "Print. The rotator cuff is a group of muscles and tendons that surround the shoulder joint, keeping the head of your upper arm bone firmly within the shallow socket of the shoulder. A rotator cuff injury can cause a dull ache in the shoulder, which often worsens when you try to sleep on the involved side.he rotator cuff is a group of muscles and tendons that surround the shoulder joint, keeping the head of your upper arm bone firmly within the shallow socket of the shoulder. The rotator cuff is a group of muscles and tendons that surround the shoulder joint, keeping the head of your upper arm bone firmly within the shallow socket of the shoulder.A rotator cuff injury can cause a dull ache in the shoulder, which often worsens when you try to sleep on the involved side.he rotator cuff is a group of muscles and tendons that surround the shoulder joint, keeping the head of y

In [15]:
dataset_train

Dataset({
    features: ['id', 'question', 'answer', 'context', 'answer_start', 'answer_end'],
    num_rows: 301763
})

In [29]:
tokenized_msmarco_train = dataset_train.map(preprocess_function, batched=True, remove_columns=dataset_train.column_names)

In [28]:
tokenized_msmarco_val=dataset_valid.map(preprocess_function, batched=True, remove_columns=dataset_valid.column_names)

In [30]:
data_collator = transformers.DefaultDataCollator()

In [31]:
model = transformers.AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [32]:
training_args = transformers.TrainingArguments(
    output_dir=models_dir,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_msmarco_train,
    eval_dataset=tokenized_msmarco_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Cloning https://huggingface.co/damapika/roberta-base_ms-marco_mod into local empty directory.
Download file pytorch_model.bin:   0%|          | 316k/473M [00:01<26:14, 315kB/s]










































































































































































Download file pytorch_model.bin: 100%|██████████| 473M/473M [00:41<00:00, 2.70MB/s] 


















Download file pytorch_model.bin: 100%|██████████| 473M/473M [00:41<00:00, 12.1MB/s]
Download file runs/May07_13-55-06_Damapika/1683460517.993868/events.out.tfevents.1683460517.Damapika.2380.1: 100%|██████████| 5.75k/5.75k [00:40<?, ?B/s]


Download file runs/May07_14-08-07_Damapika/events.out.tfevents.1683461308.Damapika.23752.0: 100%|██████████| 25.1k/25.1k [00:40<00:00, 206B/s]




Download file runs/May07_14-08-07_Damapika/1683461308.6727445/events.out.tfevents.1683461308.Damapika.23752.1: 100%|██████████| 5.75k/5.75k [00:40<?, ?B/s]






D

In [33]:
trainer.train()

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damapika. Use `wandb login --relogin` to force relogin


  1%|▏         | 500/37722 [16:12<28:34:14,  2.76s/it]

{'loss': 4.4283, 'learning_rate': 2.9602354063941467e-05, 'epoch': 0.03}


  3%|▎         | 1000/37722 [38:54<28:12:42,  2.77s/it]

{'loss': 4.1017, 'learning_rate': 2.9204708127882934e-05, 'epoch': 0.05}


...

 99%|█████████▉| 37500/37722 [8:57:44<02:28,  1.49it/s]  

{'loss': 3.4253, 'learning_rate': 1.7655479560998888e-07, 'epoch': 1.99}


                                                       
100%|██████████| 37722/37722 [9:22:17<00:00,  1.12it/s]

{'eval_loss': 3.535928964614868, 'eval_runtime': 1296.9223, 'eval_samples_per_second': 77.559, 'eval_steps_per_second': 4.848, 'epoch': 2.0}
{'train_runtime': 33743.3362, 'train_samples_per_second': 17.886, 'train_steps_per_second': 1.118, 'train_loss': 3.5997305618706013, 'epoch': 2.0}


TrainOutput(global_step=37722, training_loss=3.5997305618706013, metrics={'train_runtime': 33743.3362, 'train_samples_per_second': 17.886, 'train_steps_per_second': 1.118, 'train_loss': 3.5997305618706013, 'epoch': 2.0})

In [34]:
trainer.push_to_hub()


Upload file pytorch_model.bin: 488MB [02:38, 5.71MB/s]                            To https://huggingface.co/damapika/roberta-base_ms-marco_mod
   0adf892..e1722fe  main -> main

Upload file pytorch_model.bin: 100%|██████████| 473M/473M [02:39<00:00, 3.12MB/s]
Upload file runs/May13_10-26-56_Damapika/events.out.tfevents.1683966499.Damapika.32352.0: 100%|██████████| 16.5k/16.5k [02:39<00:00, 106B/s]  
To https://huggingface.co/damapika/roberta-base_ms-marco_mod
   e1722fe..76c582c  main -> main



'https://huggingface.co/damapika/roberta-base_ms-marco_mod/commit/e1722fefff5c4629cf75eea71e43b76643cc38d6'

In [2]:
question = "Who is Fyodor Dostoevsky?"
context = "In the world of literature, there have been many authors who have gained a reputation for their ability to create complex characters. One such author is Fyodor Dostoevsky, a Russian novelist who wrote several influential works in the 19th century."

In [3]:
question_answerer = transformers.pipeline("question-answering", model="damapika/roberta-base_ms-marco_mod")
question_answerer(question=question, context=context)

{'score': 0.026595955714583397, 'start': 246, 'end': 247, 'answer': '.'}